In [1]:
import pandas as pd
from ast import literal_eval
import numpy as np
from scipy.spatial.distance import euclidean, pdist, squareform
from os.path import join as os_join
import seaborn as sns
import matplotlib.pylab as plt
from pyvis import network as net


# pd.set_option('display.max_colwidth', None)

In [2]:
similarity_path = "../data/processed/vis_dataset/similarity_combined.csv"
dataset_path = "../data/processed/vis_dataset/vis_data.csv"

output_path = "../reports/figures/ref_similarity/similarity_using_citeTo_and_citedBy.html"

In [3]:
dataset = pd.read_csv(dataset_path)
dataset.sample(2)

,Conference,Year,Title,DOI,id,cite_to_list,cited_by_list,Link,FirstPage,LastPage,...,Abstract,AuthorNames-Deduped,AuthorNames,AuthorAffiliation,InternalReferences,AuthorKeywords,AminerCitationCount_02-2020,XploreCitationCount - 2020-01,PubsCited,Award
552,InfoVis,2010,Graphical Perception of Multiple Time Series,10.1109/TVCG.2010.162,552,"[384, 326, 71, 400, 246, 312]","[649, 791, 832, 908, 1067, 1125, 1133, 1193, 1...",http://dx.doi.org/10.1109/TVCG.2010.162,927,934,...,Line graphs have been the visualization of cho...,Waqas Javed;Bryan McDonnel;Niklas Elmqvist,Waqas Javed;Bryan McDonnel;Niklas Elmqvist,Purdue University;Purdue University;Purdue Uni...,10.1109/TVCG.2008.166;10.1109/TVCG.2007.70583;...,"Line graphs, braided graphs, horizon graphs, s...",104.0,86.0,29.0,NaN
1636,InfoVis,2019,Critical Reflections on Visualization Authorin...,10.1109/TVCG.2019.2934281,1636,[],[],http://dx.doi.org/10.1109/TVCG.2019.2934281,461,471,...,An emerging generation of visualization author...,Arvind Satyanarayan;Bongshin Lee;Donghao Ren;J...,Arvind Satyanarayan;Bongshin Lee;Donghao Ren;J...,Massachusetts Institute of Technology;Microsof...,NaN,"Critical reflection,visualization authoring,ex...",1.0,0.0,NaN,NaN


In [4]:
id_title_abstract = "<b>Paper id</b>: " + dataset["id"].astype(str) + " ("  + dataset["Conference"] + ") " " - "\
"<a href=\""  + dataset.Link + "\" target=\"_blank\">Link</a>" + \
"; <br><b>Title</b>: " + dataset["Title"] + \
"; <br><b>Abstract</b>: " + dataset["Abstract"]

map_id_to_label={k: v for  k,v in enumerate(id_title_abstract)}
map_id_to_label[0]

'<b>Paper id</b>: 0 (InfoVis)  - <a href="http://dx.doi.org/10.1109/INFVIS.1995.528680" target="_blank">Link</a>; <br><b>Title</b>: Visualisation for functional design; <br><b>Abstract</b>: We present two novel visualisation tools: the Influence Explorer and the Prosection Matrix. These were specifically created to support engineering artifact design and similar tasks in which a set of parameter values must be chosen to lead to acceptable artifact performance. These tools combine two concepts. One is the interactive and virtually immediate responsive display of data in a manner conducive to the acquisition of insight. The other, involving the precalculation of samples of artifact performance, facilitates smooth exploration and optimisation leading to a design decision. The anticipated benefits of these visualisation tools are illustrated by an example taken from electronic circuit design, in which full account must be taken of the uncertainties in parameter values arising from inevitab

In [5]:
df = pd.read_csv(similarity_path)
df

,id,0,1,2,3,4,5,6,7,8,...,1736,1737,1738,1739,1740,1741,1742,1743,1744,1745
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1741,1741,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1742,1742,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1743,1743,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1744,1744,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
arr = df.values
index_names = df.index
col_names = df.columns

#  Get indices where such threshold is crossed; avoid diagonal elems
R,C = np.where(np.triu(arr,1)>0.0)

# Arrange those in columns and put out as a dataframe
out_arr = np.column_stack((index_names[R],col_names[C],arr[R,C]))
df_out = pd.DataFrame(out_arr,columns=['from','to','weight'])
df_out["from"] = df_out["from"].astype(np.int16)
df_out["to"] = df_out["to"].astype(np.int16)
df_out

,from,to,weight
0,0,238,1.0
1,0,244,1.0
2,0,456,1.0
3,0,512,1.0
4,0,550,1.0
...,...,...,...
60213,1611,1612,2.0
60214,1611,1614,1.0
60215,1611,1615,2.0
60216,1614,1615,1.0


## Plot using Pyvis

In [7]:
color_dict = {
    "InfoVis": "blue",
    "VAST": "orange",
    "SciVis": "red"
}

def get_node_color(node, df, color_dict):
    conference = df[df.id == node].Conference.values[0]
    return color_dict[conference]

In [8]:
thres = 6
df_out_thres = df_out[df_out.weight >= thres]
plotting_all_nodes = False

file_name= output_path
g = net.Network(height='95%', width='70%',heading='Paper Similarity Using #BibliographicCouplings and #Co-citations')

if plotting_all_nodes:
    node_list = list(df.index.astype(np.int16))
else:
    node_list = list(set.union( set(df_out_thres["from"]), set(df_out_thres["to"]) ) )

node_labels = [ map_id_to_label.get(node) for node in node_list]
node_colors = [ get_node_color(node, dataset, color_dict) for node in node_list]
    
g.add_nodes(node_list, title=node_labels, label=node_list, color=node_colors)

edge_list = zip(df_out_thres["from"], df_out_thres["to"], df_out_thres['weight'])
for e in edge_list:
    g.add_edge(e[0], e[1], value = e[2], title=str(e[2])) #arrows = 'to')
    
# g.show_buttons(filter_=["physics"])
# g.show_buttons()

# g.show(file_name)

In [9]:
# !open $file_name

## Plot using Jaal

More elegant and faster, but still in its infancy though. There's a lack of some crucial features such as "edge thickness"

In [ ]:
import pandas as pd
from jaal import Jaal
from jaal.datasets import load_got


thres = 1
set_thres_nodes = df_out[df_out["weight"] >= thres]
edges = df_out_thres


edges['label'] = edges['weight'].astype(str)



set_thres_nodes = set.union( set(df_out_thres["from"]), set(df_out_thres["to"]) )
nodes = pd.DataFrame({"id": df.index})
nodes["having_edge"] = nodes["id"].map(lambda x: x in set_thres_nodes).astype(str)
nodes["conference"] = nodes["id"].map(lambda node_id: dataset[dataset["id"] == node_id ]["Conference"].values[0])
port=8050

while True:
    try:
        Jaal(edges, nodes).plot(port=port)
    except:
        port+=1


/Users/chaupham/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


Parsing the data...Done
Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


/Users/chaupham/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1787: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, val, pi)
 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [06/Apr/2021 16:05:45] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2021 16:05:45] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2021 16:05:45] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2021 16:05:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2021 16:05:45] "POST /_dash-update-component HTTP/1.1" 200 -


No trigger


127.0.0.1 - - [06/Apr/2021 16:06:31] "POST /_dash-update-component HTTP/1.1" 200 -


inside color edge label


127.0.0.1 - - [06/Apr/2021 16:06:35] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2021 16:06:37] "POST /_dash-update-component HTTP/1.1" 200 -


inside color node conference


127.0.0.1 - - [06/Apr/2021 16:06:40] "POST /_dash-update-component HTTP/1.1" 200 -
